In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[1]").appName("Examples").getOrCreate()

df = spark.read.options(header='true', inferSchema='true').csv("small_zipcode.csv")

df.printSchema()
df.show(truncate=False)

root
 |-- id: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- population: integer (nullable = true)

+---+-------+--------+-------------------+-----+----------+
|id |zipcode|type    |city               |state|population|
+---+-------+--------+-------------------+-----+----------+
|1  |704    |STANDARD|null               |PR   |30100     |
|2  |704    |null    |PASEO COSTA DEL SUR|PR   |null      |
|3  |709    |null    |BDA SAN LUIS       |PR   |3700      |
|4  |76166  |UNIQUE  |CINGULAR WIRELESS  |TX   |84000     |
|5  |76177  |STANDARD|null               |TX   |null      |
+---+-------+--------+-------------------+-----+----------+



In [2]:
df.write.option("header",True) \
        .partitionBy("state") \
        .mode("overwrite") \
        .csv("zipcodes-state1")

In [3]:
df.write.option("header",True) \
        .partitionBy("state","city") \
        .mode("overwrite") \
        .csv("zipcodes-state2")

In [6]:
df.repartition(2) \
        .write.option("header",True) \
        .partitionBy("state") \
        .mode("overwrite") \
        .csv("zipcodes-state-more")

In [8]:
#Data skew
df.write.option("header",True) \
        .option("maxRecordsPerFile", 2) \
        .partitionBy("state") \
        .mode("overwrite") \
        .csv("zipcodes-state3")

In [18]:
#read specific partition
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[1]").appName("Examples").getOrCreate()
df1=spark.read.option("header",True).csv("zipcodes-state3/state=PR")
df1.printSchema()
df1.show()

root
 |-- id: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- type: string (nullable = true)
 |-- city: string (nullable = true)
 |-- population: string (nullable = true)

+---+-------+--------+-------------------+----------+
| id|zipcode|    type|               city|population|
+---+-------+--------+-------------------+----------+
|  1|    704|STANDARD|               null|     30100|
|  2|    704|    null|PASEO COSTA DEL SUR|      null|
|  3|    709|    null|       BDA SAN LUIS|      3700|
+---+-------+--------+-------------------+----------+



In [21]:
parqDF = spark.read.option("header",True).csv("zipcodes-state3")
parqDF.createOrReplaceTempView("ZIPCODE")
spark.sql("select * from ZIPCODE  where state='PR' and city = 'BDA SAN LUIS'").show()

+---+-------+----+------------+----------+-----+
| id|zipcode|type|        city|population|state|
+---+-------+----+------------+----------+-----+
|  3|    709|null|BDA SAN LUIS|      3700|   PR|
+---+-------+----+------------+----------+-----+

